<a href="https://colab.research.google.com/github/akashsandeepa11/model-x-dementia-risk-predictor/blob/main/randomforest_with_feature_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Core libraries for data manipulation
import pandas as pd
import numpy as np

# Libraries for preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Machine Learning Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier  # A popular gradient boosting algorithm

# Libraries for evaluation
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully.")

Libraries imported successfully.


In [9]:
# --- IMPORTANT ---
# Load your dataset here.
# Change 'nacc_dataset.csv' to the path of your actual file.
try:
    df = pd.read_csv('/content/drive/MyDrive/Dementia Prediction Dataset.csv')
    print(f"Dataset loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    print("Error: Dataset file not found.")
    print("Please update the 'pd.read_csv()' line with your file path.")
    # As a placeholder, I'll create an empty DataFrame to allow subsequent cells to run
    df = pd.DataFrame()

# Display the first 5 rows to understand the data
print(df.head())

Dataset loaded successfully. Shape: (195196, 1024)
       NACCID  NACCADC PACKET  FORMVER  VISITMO  VISITDAY  VISITYR  NACCVNUM  \
0  NACC002909      186      I      3.0       12        28     2022         1   
1  NACC002909      186      F      3.0        1        23     2024         2   
2  NACC003487      186      I      3.0       11        15     2023         1   
3  NACC004352      186      I      3.0       10         5     2021         1   
4  NACC004687      186      I      3.0       11        14     2022         1   

   NACCAVST  NACCNVST  ...  NPATGAM1  NPATGAM2  NPATGAM3  NPATGAM4  NPATGAM5  \
0         2         2  ...        -4        -4        -4        -4        -4   
1         2         2  ...        -4        -4        -4        -4        -4   
2         1         1  ...        -4        -4        -4        -4        -4   
3         1         1  ...        -4        -4        -4        -4        -4   
4         1         1  ...        -4        -4        -4        -4  

In [10]:
if not df.empty:
    # 1. DEFINE YOUR TARGET VARIABLE
    TARGET_VARIABLE = 'DEMENTED'

    # 2. DEFINE THE MASTER LIST of all possible non-medical features
    ALL_POSSIBLE_FEATURES = [
        # Form A1: Subject Demographics
        'NACCAGE', 'SEX', 'EDUC', 'MARISTAT', 'NACCLIVS', 'RESIDENC', 'HANDED',
        'HISPANIC', 'RACE', 'RACESEC', 'RACETER', 'PRIMLANG', 'INDEPEND',

        # Form A2: Co-participant Demographics
        'INRELTO', 'INLIVWTH', 'INVISITS', 'INCALLS',

        # Form A3: Subject Family History
        'NACCFAM', 'NACCMOM', 'NACCDAD',

        # Form A4: Subject Medications
        'ANYMEDS',

        # Form A5: Subject Health History
        'TOBAC30', 'TOBAC100', 'SMOKYRS', 'PACKSPER', 'QUITSMOK', 'ALCOCCAS',
        'ALCFREQ', 'ALCOHOL', 'ABUSOTHR', 'CVHATT', 'CVAFIB', 'CVANGIO',
        'CVBYPASS', 'CVPACDEF', 'CVPACE', 'CVCHF', 'CVANGINA', 'CVHVALVE',
        'CVOTHR', 'HYPERTEN', 'HYPERCHO', 'CBSTROKE', 'NACCSTYR', 'CBTIA',
        'NACCTIYR', 'PD', 'PDYR', 'SEIZURES', 'NACCTBI', 'DIABETES', 'DIABTYPE',
        'B12DEF', 'THYROID', 'ARTHRIT', 'APNEA', 'RBD', 'INSOMN', 'OTHSLEEP',
        'PTSD', 'BIPOLAR', 'SCHIZ', 'DEP2YRS', 'DEPOTHR', 'ANXIETY', 'OCD',
        'INCONTU', 'INCONTF',

        # Form B1: Physical
        'HEIGHT', 'WEIGHT', 'NACCBMI', 'VISION', 'VISCORR', 'VISWCORR',
        'HEARING', 'HEARAID', 'HEARWAID',

        # Form B9: Self-Reported Decline
        'DECSUB', 'DECIN',

        # Form B7: Functional Activities
        'BILLS', 'TAXES', 'SHOPPING', 'GAMES', 'STOVE', 'MEALPREP',
        'EVENTS', 'PAYATTN', 'REMDATES', 'TRAVEL',

        # Milestones Form
        'NACCNURP',

        # Form CLS: Linguistic History (These are the ones causing the error)
        'APREFLAN', 'AYRSPAN', 'AYRENGL', 'APCSPAN', 'APCENGL',
        'NACCSPNL', 'NACCENGL'
    ]

    # 3. CRITICAL STEP: Filter the list to only include features in your CSV
    NON_MEDICAL_FEATURES = [col for col in ALL_POSSIBLE_FEATURES if col in df.columns]

    # Find and report any missing features (for your information)
    missing_from_csv = [col for col in ALL_POSSIBLE_FEATURES if col not in df.columns]

    print(f"Target variable set to: {TARGET_VARIABLE}")
    print(f"Found {len(NON_MEDICAL_FEATURES)} available non-medical features in your file.")

    if missing_from_csv:
        print("\nNote: The following features from the data dictionary were not found in your file and will be skipped:")
        print(missing_from_csv)

else:
    print("DataFrame is empty. Please load data in Cell 2.")

Target variable set to: DEMENTED
Found 90 available non-medical features in your file.

Note: The following features from the data dictionary were not found in your file and will be skipped:
['APREFLAN', 'AYRSPAN', 'AYRENGL', 'APCSPAN', 'APCENGL', 'NACCSPNL', 'NACCENGL']


In [11]:
def engineer_features(X_df):
    """
    Takes the feature DataFrame (X) and adds new engineered features.
    """
    # Make a copy to avoid changing the original data
    X_engineered = X_df.copy()

    # --- 1. Create Composite Scores (Summing related risks) ---

    # List of all functional activity columns (Form B7)
    functional_cols = [
        'BILLS', 'TAXES', 'SHOPPING', 'GAMES', 'STOVE',
        'MEALPREP', 'EVENTS', 'PAYATTN', 'REMDATES', 'TRAVEL'
    ]
    # Fill NaNs with 0 (assume 'no difficulty') before summing
    X_engineered['FUNC_DIFFICULTY_SCORE'] = X_engineered[functional_cols].fillna(0).sum(axis=1)

    # List of self-reported cardiovascular conditions (Form A5)
    cvd_cols = [
        'CVHATT', 'CVAFIB', 'CVANGIO', 'CVBYPASS', 'CVPACDEF', 'CVPACE',
        'CVCHF', 'CVANGINA', 'CVHVALVE', 'HYPERTEN', 'HYPERCHO', 'CBSTROKE', 'CBTIA'
    ]
    # Fill NaNs with 0 (assume 'absent') before summing to get a risk count
    X_engineered['CVD_RISK_COUNT'] = X_engineered[cvd_cols].fillna(0).sum(axis=1)

    # List of self-reported psychiatric conditions (Form A5)
    psych_cols = [
        'PTSD', 'BIPOLAR', 'SCHIZ', 'DEP2YRS', 'DEPOTHR', 'ANXIETY', 'OCD'
    ]
    X_engineered['PSYCH_RISK_COUNT'] = X_engineered[psych_cols].fillna(0).sum(axis=1)


    # --- 2. Bin Numerical Features (Turning numbers into categories) ---

    # Bin BMI into standard categories
    bmi_bins = [0, 18.5, 24.9, 29.9, 100]
    bmi_labels = ['Underweight', 'Normal', 'Overweight', 'Obese']
    X_engineered['BMI_CATEGORY'] = pd.cut(X_engineered['NACCBMI'], bins=bmi_bins, labels=bmi_labels, right=False)

    # Bin Age into groups
    age_bins = [0, 65, 75, 85, 120]
    age_labels = ['65_or_less', '66-75', '76-85', '86_and_up']
    X_engineered['AGE_GROUP'] = pd.cut(X_engineered['NACCAGE'], bins=age_bins, labels=age_labels, right=False)


    # --- 3. Create Interaction Features ---

    # Check interaction between age and education
    # (The pipeline will handle imputing NaNs in the original NACCAGE/EDUC columns)
    X_engineered['AGE_X_EDUC'] = X_engineered['NACCAGE'] * X_engineered['EDUC']

    print("Feature engineering complete.")
    print(f"Original features: {len(X_df.columns)}, New features: {len(X_engineered.columns)}")

    return X_engineered

In [13]:
if not df.empty:
    # 1. --- Select X and y ---
    df_clean = df.dropna(subset=[TARGET_VARIABLE])

    # NON_MEDICAL_FEATURES should be available from running Cell 3
    X = df_clean[NON_MEDICAL_FEATURES]
    y = df_clean[TARGET_VARIABLE].astype(int)

    # 2. --- Data Cleaning ---
    MISSING_CODES = [
        -4, 8, 9, 88, 99, 888, 999, 8888, 9999,
        95, 96, 97, 98, 995, 996, 997, 998
    ]
    X = X.replace(MISSING_CODES, np.nan)
    print(f"Missing values (as np.nan) identified: {X.isna().sum().sum()}")

    # 3. --- Apply Feature Engineering ---
    # This calls the function from Cell 4
    X_engineered = engineer_features(X)

    # 4. --- Define Preprocessing Pipelines (with new features) ---

    # --- FIX IS HERE ---
    # This list needs to be defined within this cell
    ALL_POSSIBLE_NUMERIC_FEATURES = [
        'NACCAGE', 'EDUC', 'SMOKYRS', 'PACKSPER', 'QUITSMOK', 'NACCSTYR',
        'NACCTIYR', 'PDYR', 'HEIGHT', 'WEIGHT', 'NACCBMI', 'AYRSPAN',
        'AYRENGL', 'APCSPAN', 'APCENGL'
    ]
    # --- END FIX ---

    # Get the original feature lists
    NUMERIC_FEATURES_ORIGINAL = [col for col in NON_MEDICAL_FEATURES if col in ALL_POSSIBLE_NUMERIC_FEATURES]
    CATEGORICAL_FEATURES_ORIGINAL = [col for col in NON_MEDICAL_FEATURES if col not in NUMERIC_FEATURES_ORIGINAL]

    # Add our new features to the lists for preprocessing
    NUMERIC_FEATURES = NUMERIC_FEATURES_ORIGINAL + [
        'FUNC_DIFFICULTY_SCORE', 'CVD_RISK_COUNT', 'PSYCH_RISK_COUNT', 'AGE_X_EDUC'
    ]
    CATEGORICAL_FEATURES = CATEGORICAL_FEATURES_ORIGINAL + [
        'BMI_CATEGORY', 'AGE_GROUP'
    ]

    # Remove duplicates just in case
    NUMERIC_FEATURES = list(set([col for col in NUMERIC_FEATURES if col in X_engineered.columns]))
    CATEGORICAL_FEATURES = list(set([col for col in CATEGORICAL_FEATURES if col in X_engineered.columns]))

    print(f"Total numeric features for pipeline: {len(NUMERIC_FEATURES)}")
    print(f"Total categorical features for pipeline: {len(CATEGORICAL_FEATURES)}")

    # Create the pipelines (same as before)
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Combine pipelines
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, NUMERIC_FEATURES),
            ('cat', categorical_transformer, CATEGORICAL_FEATURES)
        ],
        remainder='drop' # Drop any original columns we didn't use
    )

    print("Preprocessing pipelines updated with new features.")

    # 5. --- Split the Data ---
    # We split the *engineered* data
    X_train, X_test, y_train, y_test = train_test_split(
        X_engineered, y,
        test_size=0.2,
        random_state=42,
        stratify=y
    )

    print(f"\nData split into training and testing sets.")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")

else:
    print("DataFrame is empty. Please load data in Cell 2.")

Missing values (as np.nan) identified: 6850211
Feature engineering complete.
Original features: 90, New features: 96
Total numeric features for pipeline: 15
Total categorical features for pipeline: 81
Preprocessing pipelines updated with new features.

Data split into training and testing sets.
X_train shape: (156156, 96)
X_test shape: (39040, 96)


In [14]:
# Logistic Regression (Baseline)

if not df.empty:
    # Create the full model pipeline
    lr_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(random_state=42, max_iter=1000))
    ])

    # --- Train the model ---
    print("Training Logistic Regression model on ENGINEERED features...")
    lr_pipeline.fit(X_train, y_train)

    # --- Evaluate the model ---
    print("\n--- Logistic Regression (Engineered) Evaluation ---")
    y_pred_lr = lr_pipeline.predict(X_test)
    y_proba_lr = lr_pipeline.predict_proba(X_test)[:, 1]

    print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
    print(f"AUC Score: {roc_auc_score(y_test, y_proba_lr):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_lr, target_names=["Not at risk (0)", "At risk (1)"]))

else:
    print("DataFrame is empty. Please load data in Cell 2 or 5.")

Training Logistic Regression model on ENGINEERED features...

--- Logistic Regression (Engineered) Evaluation ---
Accuracy: 0.9288
AUC Score: 0.9754

Classification Report:
                 precision    recall  f1-score   support

Not at risk (0)       0.94      0.96      0.95     27522
    At risk (1)       0.90      0.86      0.88     11518

       accuracy                           0.93     39040
      macro avg       0.92      0.91      0.91     39040
   weighted avg       0.93      0.93      0.93     39040



In [15]:
# Random Forest Classifier

if not df.empty:
    # Create the full model pipeline
    rf_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))
    ])

    # --- Train the model ---
    print("Training Random Forest model on ENGINEERED features...")
    rf_pipeline.fit(X_train, y_train)

    # --- Evaluate the model ---
    print("\n--- Random Forest (Engineered) Evaluation ---")
    y_pred_rf = rf_pipeline.predict(X_test)
    y_proba_rf = rf_pipeline.predict_proba(X_test)[:, 1]

    print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
    print(f"AUC Score: {roc_auc_score(y_test, y_proba_rf):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_rf, target_names=["Not at risk (0)", "At risk (1)"]))

else:
    print("DataFrame is empty. Please load data in Cell 2 or 5.")

Training Random Forest model on ENGINEERED features...

--- Random Forest (Engineered) Evaluation ---
Accuracy: 0.9342
AUC Score: 0.9788

Classification Report:
                 precision    recall  f1-score   support

Not at risk (0)       0.95      0.96      0.95     27522
    At risk (1)       0.90      0.88      0.89     11518

       accuracy                           0.93     39040
      macro avg       0.92      0.92      0.92     39040
   weighted avg       0.93      0.93      0.93     39040



In [16]:
# XGBoost Classifier

if not df.empty:
    # Create the full model pipeline
    xgb_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
    ])

    # --- Train the model ---
    print("Training XGBoost model on ENGINEERED features...")
    xgb_pipeline.fit(X_train, y_train)

    # --- Evaluate the model ---
    print("\n--- XGBoost (Engineered) Evaluation ---")
    y_pred_xgb = xgb_pipeline.predict(X_test)
    y_proba_xgb = xgb_pipeline.predict_proba(X_test)[:, 1]

    print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
    print(f"AUC Score: {roc_auc_score(y_test, y_proba_xgb):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_xgb, target_names=["Not at risk (0)", "At risk (1)"]))

else:
    print("DataFrame is empty. Please load data in Cell 2 or 5.")

Training XGBoost model on ENGINEERED features...

--- XGBoost (Engineered) Evaluation ---
Accuracy: 0.9337
AUC Score: 0.9792

Classification Report:
                 precision    recall  f1-score   support

Not at risk (0)       0.95      0.96      0.95     27522
    At risk (1)       0.89      0.88      0.89     11518

       accuracy                           0.93     39040
      macro avg       0.92      0.92      0.92     39040
   weighted avg       0.93      0.93      0.93     39040



In [17]:
if not df.empty:
    print("--- Model Performance Summary (with Engineered Features) ---")

    results_engineered = {
        'Model': ['Logistic Regression (Eng)', 'Random Forest (Eng)', 'XGBoost (Eng)'],
        'Accuracy': [
            accuracy_score(y_test, y_pred_lr),
            accuracy_score(y_test, y_pred_rf),
            accuracy_score(y_test, y_pred_xgb)
        ],
        'AUC Score': [
            roc_auc_score(y_test, y_proba_lr),
            roc_auc_score(y_test, y_proba_rf),
            roc_auc_score(y_test, y_proba_xgb)
        ],
        'F1-Score (At risk)': [
            f1_score(y_test, y_pred_lr, pos_label=1),
            f1_score(y_test, y_pred_rf, pos_label=1),
            f1_score(y_test, y_pred_xgb, pos_label=1)
        ]
    }

    results_engineered_df = pd.DataFrame(results_engineered)
    print(results_engineered_df.to_markdown(index=False, floatfmt=".4f"))

    print("\n\nCompare this table to your first run. Did the new features help?")
    print("Your next steps are still to tune the best model (Cell 10) and check explainability (Cell 12).")

else:
    print("DataFrame is empty. Please run the model cells first.")

--- Model Performance Summary (with Engineered Features) ---
| Model                     |   Accuracy |   AUC Score |   F1-Score (At risk) |
|:--------------------------|-----------:|------------:|---------------------:|
| Logistic Regression (Eng) |     0.9288 |      0.9754 |               0.8768 |
| Random Forest (Eng)       |     0.9342 |      0.9788 |               0.8874 |
| XGBoost (Eng)             |     0.9337 |      0.9792 |               0.8868 |


Compare this table to your first run. Did the new features help?
Your next steps are still to tune the best model (Cell 10) and check explainability (Cell 12).


In [18]:
from sklearn.model_selection import GridSearchCV

if not df.empty:
    print("Starting Hyperparameter Tuning for Random Forest...")

    # 1. Re-create the pipeline
    # We use the 'rf_pipeline' variable name from Cell 7
    rf_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42))
    ])

    # 2. Define the 'parameter grid' to search
    # These are common parameters to tune for a Random Forest
    param_grid_rf = {
        'classifier__n_estimators': [100, 200],         # Number of trees
        'classifier__max_depth': [5, 10, None],          # Max depth of each tree (None = no limit)
        'classifier__min_samples_leaf': [1, 2]           # Min samples required at a leaf node
    }

    # 3. Set up the Grid Search
    # We'll name this 'grid_search_rf' to avoid overwriting the XGBoost one
    grid_search_rf = GridSearchCV(
        estimator=rf_pipeline,
        param_grid=param_grid_rf,
        scoring='roc_auc',  # Optimize for the best AUC score
        cv=3,               # 3-fold cross-validation
        verbose=1,          # Print updates
        n_jobs=-1           # Use all available CPU cores
    )

    # 4. Run the Grid Search (This will take a few minutes)
    grid_search_rf.fit(X_train, y_train)

    # 5. Show the results
    print("\n--- Tuning Complete (Random Forest) ---")
    print(f"Best AUC Score found: {grid_search_rf.best_score_:.4f}")
    print("Best parameters found:")
    print(grid_search_rf.best_params_)

else:
    print("DataFrame is empty. Please run Cell 5 first.")

Starting Hyperparameter Tuning for Random Forest...
Fitting 3 folds for each of 12 candidates, totalling 36 fits

--- Tuning Complete (Random Forest) ---
Best AUC Score found: 0.9775
Best parameters found:
{'classifier__max_depth': None, 'classifier__min_samples_leaf': 2, 'classifier__n_estimators': 200}


In [19]:
if not df.empty and 'grid_search_rf' in locals():
    print("--- Evaluating the BEST Tuned Random Forest on Test Data ---")

    # Get the best model found by the grid search
    best_rf_model = grid_search_rf.best_estimator_

    # --- Evaluate the model ---
    y_pred_best_rf = best_rf_model.predict(X_test)
    y_proba_best_rf = best_rf_model.predict_proba(X_test)[:, 1]

    print(f"Accuracy: {accuracy_score(y_test, y_pred_best_rf):.4f}")
    print(f"AUC Score: {roc_auc_score(y_test, y_proba_best_rf):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_best_rf, target_names=["Not at risk (0)", "At risk (1)"]))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred_best_rf))

else:
    print("Please run Cell 10 to complete the Random Forest grid search first.")

--- Evaluating the BEST Tuned Random Forest on Test Data ---
Accuracy: 0.9336
AUC Score: 0.9792

Classification Report:
                 precision    recall  f1-score   support

Not at risk (0)       0.95      0.96      0.95     27522
    At risk (1)       0.89      0.88      0.89     11518

       accuracy                           0.93     39040
      macro avg       0.92      0.92      0.92     39040
   weighted avg       0.93      0.93      0.93     39040


Confusion Matrix:
[[26323  1199]
 [ 1394 10124]]
